In [ ]:
#Setup
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
import torch
import os
from IPython.display import Image, clear_output


Cloning into 'yolov5'...
remote: Enumerating objects: 16026, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 16026 (delta 33), reused 40 (delta 25), pack-reused 15967
Receiving objects: 100% (16026/16026), 14.67 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (10995/10995), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [ ]:

os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="3kZUcOURZwpHNUXrVRYO")
project = rf.workspace("cs473-proj").project("cs473-proj1")
dataset = project.version(2).download("yolov5")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstall

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /content/datasets/CS473-Proj1-2 in yolov5pytorch:: 100%|██████████| 779/779 [00:00<00:00, 1843.53it/s]


In [ ]:
#Train model using yolo
!python train.py --img 640 --batch 16 --epochs 100 --data /content/datasets/CS473-Proj1-2/data.yaml --cfg yolov5m.yaml --weights yolov5m.pt --name step1_train


In [ ]:
!zip -r /content/meeting_weights.zip /content/yolov5/runs/train/step1_train/weights

In [ ]:
#Prediction (for new data sets);
!python detect.py --source /content/testing/images --weights /content/yolov5/runs/train/step1_train/weights/best.pt --img-size 640 --conf-thres 0.25 --save-txt

In [ ]:
# create the function to get the co-ordinates
import os
from IPython.display import Image, display

# Define the directory where the inference results are saved
output_directory = "runs/detect/exp3"

# Get a list of all the image files in the output directory
image_files = [f for f in os.listdir(output_directory) if os.path.isfile(os.path.join(output_directory, f)) and f.endswith(('.png', '.jpg', '.jpeg'))]

# Display each image
for image_file in image_files:
    display(Image(filename=os.path.join(output_directory, image_file)))


In [ ]:
# Cropping images
!pip install PyYAML
import yaml
import os
import cv2
import torch

# Directory where inference results are saved
output_directory = "runs/detect/exp3"
labels_file = os.path.join(output_directory, "labels")  # directory containing .txt files with detection results
images_file = os.path.join(output_directory, "images")  # directory containing images with drawn bounding boxes

# Load class names from YOLOv5's data.yaml
with open("/content/datasets/CS473-Proj1-2/data.yaml", 'r') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
class_names = data['names']

# Dictionary to store cropped images for each class
cropped_images_dict = {class_name: [] for class_name in class_names}

# For each image, load its corresponding labels and crop detected regions
for filename in os.listdir("/content/yolov5/"+labels_file):
    if filename.endswith('.txt'):
        image_path = os.path.join(images_file, filename.replace('.txt', '.jpg'))
        image = cv2.imread(image_path)

        with open(os.path.join("/content/yolov5/"+labels_file, filename), 'r') as f:
            for line in f.readlines():
                class_id, x_center, y_center, width, height = map(float, line.split())
                class_id = int(class_id)
                class_name = class_names[class_id]

                # Convert YOLO box format to pixel values
                x_center, y_center, width, height = x_center * image.shape[1], y_center * image.shape[0], width * image.shape[1], height * image.shape[0]
                x1, y1, x2, y2 = int(x_center - width/2), int(y_center - height/2), int(x_center + width/2), int(y_center + height/2)

                cropped_img = image[y1:y2, x1:x2]
                cropped_images_dict[class_name].append(cropped_img)

# Now, cropped_images_dict contains lists of cropped images for each class.
